In [260]:
import sys
print (sys.version)

3.7.2 (v3.7.2:9a3ffc0492, Dec 24 2018, 02:44:43) 
[Clang 6.0 (clang-600.0.57)]


In [261]:
#https://stackoverflow.com/questions/47944782/r-data-scraping-crawling-with-dynamic-multiple-urls
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re
import random

In [262]:
import locale, datetime #for stringparsing in german
locale.setlocale(locale.LC_ALL, "de_de")
#locale.setlocale(locale.LC_ALL, "fr_FR")

'de_de'

**Extracting the index leading to mandate**

In [263]:
#highlight_docid=atf%3A%2F%2F95-IV-162
queryKeyword = "mord"
queryRank = "1"
#bgerQuery2 = "https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=1954&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words="+ queryKeyword + "&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank="+queryRank+"&highlight_docid=atf%3A%2F%2F95-IV-162%3Ade&number_of_ranks=945&azaclir=clir"

In [25]:
#bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year=1954&to_year=2019'

In [264]:
from_year = "2017"
to_year = "2019"

bgerQuery2 = 'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words='+ queryKeyword +'&lang=de&top_subcollection_clir=bge&from_year='+ from_year + '&to_year=' + to_year

In [265]:
bgerQuery2

'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=simple_query&query_words=mord&lang=de&top_subcollection_clir=bge&from_year=2017&to_year=2019'

In [266]:
#check if content was loaded properly
bgerQuery2 = requests.get(bgerQuery2)
bgerQuery2

<Response [200]>

In [8]:
#bgerQuery2.content

In [9]:
#soup = BeautifulSoup(page.content, 'html.parser')

In [267]:
bgerSoup = BeautifulSoup(bgerQuery2.content, 'html.parser')

In [268]:
bgerSoup


<!DOCTYPE html>

<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width initial-scale=1" name="viewport"/>
<link href="/modules/tfl-internet-templates/images/favicon.ico" rel="icon"/>
<title>Eurospider Suche: mord</title>
<link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/master.css" rel="stylesheet" title="Eurospider Default Screen Style" type="text/css"/>
<link href="/modules/tfl-internet-templates/css/styles.css" media="screen" rel="stylesheet" type="text/css"/>
<style media="print" type="text/css">
        @media print{
            .body .size1of3{
                display: none;
            }
            .head .grid_left{
                display: none;
            }
            .access-a, .form-search, .define-admin {
                display: none !important;
            }
            body{
                font-family: Arial, sans-serif;
                font-size: 12px !important;
       

In [269]:
links = bgerSoup.find_all(class_='rank_title')
linksList = []
for a in links:
    linksList.append(a.find('a')['href'])
    #print(a)
print(linksList[0])

https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=1&highlight_docid=atf%3A%2F%2F144-IV-285%3Ade&number_of_ranks=8&azaclir=clir


**Make a new query with Linkslist (only based on first entry)**

In [270]:
mBger = requests.get(linksList[0])

In [33]:
#mBger.content

In [271]:
mBger = BeautifulSoup(mBger.content, 'html.parser')

In [290]:
mBger


<!DOCTYPE html>

<html lang="de">
<head>
<meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
<meta content="width=device-width initial-scale=1" name="viewport"/>
<link href="/modules/tfl-internet-templates/images/favicon.ico" rel="icon"/>
<title>144 IV 285</title>
<link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/master.css" rel="stylesheet" type="text/css"/><!-- do not add a attribute 'title' here, as Firefox will stop working for media print! -->
<link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/print.css" media="print" rel="stylesheet" type="text/css"/>
<!--[if IE 6]><link href="https://www.bger.ch/ext/eurospider/live/de/php/clir/http/css/ie_win_pos_abs.css" rel="stylesheet" type="text/css" media="screen"><![endif]-->
<link href="/modules/tfl-internet-templates/css/styles.css" media="screen" rel="stylesheet" type="text/css"/>
<style media="print" type="text/css">
        @media print{
            .body .size1of3{
      

**display children / find correct html elements**

In [272]:
len(list(mBger.children))
#find body
body = list(mBger.children)[12]
len(list(body.children))
#body

33

**get all required variables from verdict**

**dateparser for string dates**

In [273]:
text2 = "41. Urteil des Kassationshofes vom 5. octobre 1969 i.S. Staatsanwaltschaft des Kantons Aargau gegen Brunner."

def parseDate(text):
    pattern = re.compile(r"""
        \d\d?   # one or two digits
        \.?
        \s?      # \s for space - one space after
        [a-z,ä,ö,ü]+  # at least one+ ascii letters (ignore case is use)set of characters that you wish to match / since + means ‘one or more repetitions’
        \s?      # one space after
        \d{4}   # four decimal digits = \d (year)
    """,re.IGNORECASE|re.VERBOSE)
    if (pattern.search(text)!= None):
        #print(pattern.search(text).group(), type(pattern.search(text).group()))
        dateStr = pattern.search(text).group()
        dateStr = dateStr.replace(".", "")
        #print(dateStr)
        #dateStr = dateStr.format('DD MMMM YYYY', locale='de')
        dateStr = detectLanguage(dateStr)
        #print(dateStr)
        #dateStr = dateStr.strftime('%m/%d/%Y')
        return dateStr
    else:
        return 0 #make 0 if date can't be found ig not existent or different language

In [274]:
from textblob import TextBlob
from langdetect import detect
import arrow

In [275]:
dateTest = '2 Dezember 2011'
def detectLanguage2(x):
 if (TextBlob(x).detect_language() == "de"):
     x = arrow.get(x, 'D MMMM YYYY', locale='de')
     x.format('YYYY-MMMM-DD')
     #print("german",x)
     return x
 elif (TextBlob(x).detect_language() == "fr"):
     x = arrow.get(x, 'D MMMM YYYY', locale='fr')
     x.format('YYYY-MMMM-DD')
     #print("french",x)
     return x
 elif (TextBlob(x).detect_language() == "it"):
     x = arrow.get(x, 'D MMMM YYYY', locale='it')
     x.format('YYYY-MMMM-DD')
     #print("2",x)
     return x
 else:
     x=0
     #print("3", x)
     return x

In [276]:
detect("4 februar 2018")

'sv'

In [277]:
def detectLanguage(string):
    months_de = ["januar", "februar", "märz", "april", "mai","juni","juli", "august", "september", "oktober", "november", "dezember"]
    months_fr = ["janvier", "février", "mars", "avril", "juin", "juillet", "août", "septembre", "octobre", "novembre", "décembre"]
    months_it = ["gennaio", "febbraio", "marzo", "aprile", "maggio", "giugno", "luglio", "agosto", "settembre", "ottobre", "novembre"," dicembre"]
    if any(x in string.lower() for x in months_de):
        string = arrow.get(string, 'D MMMM YYYY', locale='de')
        #string.format('YYYY-MMMM-DD')
        string = string.strftime('%Y-%m-%d')
        return string
    elif any(x in string.lower() for x in months_fr):
        string = arrow.get(string, 'D MMMM YYYY', locale='fr')
        string = string.strftime('%Y-%m-%d')
        #string.format('YYYY-MMMM-DD')
        #print("french",x)
        return string
    elif any(x in string.lower() for x in months_it):
        string = arrow.get(string, 'D MMMM YYYY', locale='it')
        string = string.strftime('%Y-%m-%d')
        #print("french",x)
        return string
    else:
        string = 0
        return string
detectLanguage(dateTest)

'2011-12-02'

In [278]:
parseDate('16. September 2003')

'2003-09-16'

**Get NAME, DATE, SUBLINKS of MAIN MANDATE**

In [386]:
def cleanString(string):
    if string[len(string)-1] == 'S':
        string = string[:-1]
    string = string.rstrip()
    string = string[4:]
    return string

In [363]:
def getDateName(link):
    #nodes = []
    r = requests.get(link)
    linkSoup = BeautifulSoup(r.content, 'html.parser')
    body = list(linkSoup.children)[12]
    #ID of mandate
    mID = cleanString(body.find_all('div', class_="center pagebreak")[0].get_text()[:14])
    #print(mID)
    mDate = body.find_all(class_="paraatf")[1].get_text()
    #quickfix, skip all dates that can't be parsed
    mDateParsed = parseDate(mDate)
    mRefs = body.find_all(class_="bgeref_id")
    mRefsLinks = extractLinks(mRefs)
    mRefsNames = listing(mRefs)
    subEdges = getEdges(mRefsNames, mID)
    #print(mRefsLinks)
    return mID, mDateParsed, mRefsLinks, subEdges, mRefs

In [387]:
mandate1 = getDateName(linksList[0])

In [303]:
linksList[0]

'https://www.bger.ch/ext/eurospider/live/de/php/clir/http/index.php?lang=de&type=highlight_simple_query&page=1&from_date=&to_date=&from_year=2017&to_year=2019&sort=relevance&insertion_date=&from_date_push=&top_subcollection_clir=bge&query_words=mord&part=all&de_fr=&de_it=&fr_de=&fr_it=&it_de=&it_fr=&orig=&translation=&rank=1&highlight_docid=atf%3A%2F%2F144-IV-285%3Ade&number_of_ranks=8&azaclir=clir'

In [412]:
nodes = []
nodes.append({"id": mandate1[0],"date":mandate1[1],"relevancy":10, "level": 1})

In [413]:
nodes

[{'id': '144 IV 285', 'date': '2018-08-15', 'relevancy': 10, 'level': 1}]

**Make a list of all links for MAIN**

**Get NAMES, DATE and SUBLINKS of each LINKED MANDATES in list (PROBLEM: many are in french & dateparser return 0)**

In [306]:
def listing(elementlist):
    ar = []
    for item in elementlist:
        #print(item.get_text())
        ar.append(item.get_text())
    #print(ar)
    return ar

In [391]:
def getEdges(linkNames, ID):
    edgeList = []
    if linkNames: #list has items
        #print("links list is not empty", linkNames)
        for i, val in enumerate(linkNames):
            #print(i)
            val = cleanString(val)
            oj = {"source": ID, "target": val}
            edgeList.append(oj)
    return edgeList
#links_main

In [352]:
#mandate1[4]

In [390]:
#names of referenced mandates
mRefs = listing(mandate1[4])
mRefs

['BGE 144 IV 1',
 'BGE 141 IV 155',
 'BGE 140 IV 57',
 'BGE 141 IV 155',
 'BGE 129 IV 305',
 'BGE 144 IV 1',
 'BGE 141 IV 155',
 'BGE 140 IV 57',
 'BGE 138 IV 1',
 'BGE 136 IV 4',
 'BGE 129 II 453',
 'BGE 141 IV 155',
 'BGE 138 IV 1',
 'BGE 136 IV 4',
 'BGE 144 IV 1',
 'BGE 137 IV 79',
 'BGE 125 IV 4',
 'BGE 120 IV 365',
 'BGE 141 IV 155',
 'BGE 137 IV 305',
 'BGE 125 IV 4',
 'BGE 132 III 315',
 'BGE 132 III 305',
 'BGE 139 IV 209',
 'BGE 143 II 661',
 'BGE 143 III 242',
 'BGE 142 IV 237',
 'BGE 137 IV 79',
 'BGE 137 IV 79',
 'BGE 137 IV 79',
 'BGE 142 IV 383']

these are the main lists we will append everything to: "nodes" and "edges"

In [414]:
edges = getEdges(mRefs,mandate1[0])

In [373]:
len(edges)

31

In [415]:
edges

[{'source': '144 IV 285', 'target': '144 IV 1'},
 {'source': '144 IV 285', 'target': '141 IV 155'},
 {'source': '144 IV 285', 'target': '140 IV 57'},
 {'source': '144 IV 285', 'target': '141 IV 155'},
 {'source': '144 IV 285', 'target': '129 IV 305'},
 {'source': '144 IV 285', 'target': '144 IV 1'},
 {'source': '144 IV 285', 'target': '141 IV 155'},
 {'source': '144 IV 285', 'target': '140 IV 57'},
 {'source': '144 IV 285', 'target': '138 IV 1'},
 {'source': '144 IV 285', 'target': '136 IV 4'},
 {'source': '144 IV 285', 'target': '129 II 453'},
 {'source': '144 IV 285', 'target': '141 IV 155'},
 {'source': '144 IV 285', 'target': '138 IV 1'},
 {'source': '144 IV 285', 'target': '136 IV 4'},
 {'source': '144 IV 285', 'target': '144 IV 1'},
 {'source': '144 IV 285', 'target': '137 IV 79'},
 {'source': '144 IV 285', 'target': '125 IV 4'},
 {'source': '144 IV 285', 'target': '120 IV 365'},
 {'source': '144 IV 285', 'target': '141 IV 155'},
 {'source': '144 IV 285', 'target': '137 IV 305'},

In [394]:
def extractLinks(htmlElements):
    array = []
    for a in htmlElements:
        #print(a)
        if a['href'] not in array: #make sure to not add duplicate mandates
            array.append(a['href'])
    return array

In [407]:
linksLevel2 = mandate1[2]

In [416]:
linksLevel3= []
for val in linksLevel2: #level 2
    #print(i,val)
    mandate2 = getDateName(val)
    #check if mandate already exists at date isn't 0 not sure if working. will edge connections be skipped? 
    if (mandate2[1] != 0 and mandate2[0] not in nodes):
        #print("jump to next level", mandate2[1])
        #print("next edgelist", mandate2[3], mandate2[1],mandate2[0])
        oj = {"id": mandate2[0],"date":mandate2[1],"relevancy":random.randint(1,9), "level": 2}
        nodes.append(oj)
        linksLevel3.append(mandate2[2])
        #print("mandate2,edges",mandate2[3])
        #edges + mandate2[3]
        edges.extend(mandate2[3])

In [417]:
nodes

[{'id': '144 IV 285', 'date': '2018-08-15', 'relevancy': 10, 'level': 1},
 {'id': '144 IV 1', 'date': '2017-10-24', 'relevancy': 5, 'level': 2},
 {'id': '141 IV 155', 'date': '2015-02-25', 'relevancy': 5, 'level': 2},
 {'id': '140 IV 57', 'date': '2014-03-06', 'relevancy': 8, 'level': 2},
 {'id': '129 IV 305', 'date': '2003-09-16', 'relevancy': 8, 'level': 2},
 {'id': '136 IV 4', 'date': '2010-01-12', 'relevancy': 2, 'level': 2},
 {'id': '129 II 453', 'date': '2003-10-27', 'relevancy': 4, 'level': 2},
 {'id': '137 IV 79', 'date': '2011-01-25', 'relevancy': 7, 'level': 2},
 {'id': '137 IV 305', 'date': '2011-08-22', 'relevancy': 5, 'level': 2},
 {'id': '132 III 31', 'date': '2006-03-01', 'relevancy': 9, 'level': 2},
 {'id': '132 III 30', 'date': '2006-02-06', 'relevancy': 3, 'level': 2},
 {'id': '139 IV 209', 'date': '2013-04-18', 'relevancy': 3, 'level': 2},
 {'id': '143 II 661', 'date': '2017-10-05', 'relevancy': 8, 'level': 2},
 {'id': '143 III 24', 'date': '2017-06-28', 'relevancy':

In [418]:
linksLevel3= sum(linksLevel3,[])
for val in linksLevel3: #level 3
    #print(val)
    mandate3 = getDateName(val)
    #print(mandate3[1])
    if (mandate3[1] != 0 and mandate3[0] not in nodes):
        #print(mandate3[1] != 0)
        oj = {"id": mandate3[0],"date":mandate3[1],"relevancy":random.randint(1,6), "level": 3}
        print(oj)
        nodes.append(oj)
        #
        #edges + mandate3[3]

{'id': '142 IV 383', 'date': '2016-09-22', 'relevancy': 3, 'level': 3}
{'id': '141 IV 155', 'date': '2015-02-25', 'relevancy': 3, 'level': 3}
{'id': '140 IV 57', 'date': '2014-03-06', 'relevancy': 6, 'level': 3}
{'id': '139 IV 209', 'date': '2013-04-18', 'relevancy': 3, 'level': 3}
{'id': '137 IV 305', 'date': '2011-08-22', 'relevancy': 5, 'level': 3}
{'id': '141 IV 305', 'date': '2015-06-23', 'relevancy': 2, 'level': 3}
{'id': '137 IV 79', 'date': '2011-01-25', 'relevancy': 6, 'level': 3}
{'id': '130 III 24', 'date': '2004-01-29', 'relevancy': 1, 'level': 3}
{'id': '138 III 49', 'date': '2012-05-16', 'relevancy': 1, 'level': 3}
{'id': '141 IV 317', 'date': '2015-06-23', 'relevancy': 2, 'level': 3}
{'id': '137 IV 305', 'date': '2011-08-22', 'relevancy': 5, 'level': 3}
{'id': '129 IV 107', 'date': '2002-11-21', 'relevancy': 3, 'level': 3}
{'id': '129 IV 305', 'date': '2003-09-16', 'relevancy': 2, 'level': 3}
{'id': '137 IV 79', 'date': '2011-01-25', 'relevancy': 1, 'level': 3}
{'id': '1

In [87]:
len(nodes)

15

In [95]:
len(nodes)

176

In [337]:
len(nodes)

89

In [338]:
len(edges)

249

In [253]:
edges

[{'source': 'BGE 144 IV 28', 'target': 'BGE 144 IV 1'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 141 IV 155'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 140 IV 57'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 141 IV 155'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 129 IV 305'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 144 IV 1'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 141 IV 155'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 140 IV 57'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 138 IV 1'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 136 IV 4'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 129 II 453'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 141 IV 155'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 138 IV 1'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 136 IV 4'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 144 IV 1'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 137 IV 79'},
 {'source': 'BGE 144 IV 28', 'target': 'BGE 125 IV 4'},
 {'source': 'BGE 144 IV 28', 'tar

In [419]:
data = {"nodes": nodes,
       "links": edges
       }

In [420]:
data

{'nodes': [{'id': '144 IV 285',
   'date': '2018-08-15',
   'relevancy': 10,
   'level': 1},
  {'id': '144 IV 1', 'date': '2017-10-24', 'relevancy': 5, 'level': 2},
  {'id': '141 IV 155', 'date': '2015-02-25', 'relevancy': 5, 'level': 2},
  {'id': '140 IV 57', 'date': '2014-03-06', 'relevancy': 8, 'level': 2},
  {'id': '129 IV 305', 'date': '2003-09-16', 'relevancy': 8, 'level': 2},
  {'id': '136 IV 4', 'date': '2010-01-12', 'relevancy': 2, 'level': 2},
  {'id': '129 II 453', 'date': '2003-10-27', 'relevancy': 4, 'level': 2},
  {'id': '137 IV 79', 'date': '2011-01-25', 'relevancy': 7, 'level': 2},
  {'id': '137 IV 305', 'date': '2011-08-22', 'relevancy': 5, 'level': 2},
  {'id': '132 III 31', 'date': '2006-03-01', 'relevancy': 9, 'level': 2},
  {'id': '132 III 30', 'date': '2006-02-06', 'relevancy': 3, 'level': 2},
  {'id': '139 IV 209', 'date': '2013-04-18', 'relevancy': 3, 'level': 2},
  {'id': '143 II 661', 'date': '2017-10-05', 'relevancy': 8, 'level': 2},
  {'id': '143 III 24', 'd

**Save variables in dynamic dictionary**

In [ ]:
#l=['mID','mRefs','mRefsLinked','links','date']
#objs = [mID,mRefs, mRefsLinked, linksListref, mDate]
l=['mID','mRefs','mRefsLinked','date']
objs = [mID,mRefs, mRefsLinked, mDate]
mDict = {}
for i, val in enumerate(l):
    k= objs[i]
    mDict[val]=k
print(mDict, type(mDict))

In [ ]:
mRefsLinked.append(mID)
mRefsLinked

In [ ]:
#https://stackoverflow.com/questions/13575090/construct-pandas-dataframe-from-items-in-nested-dictionary
#pd.DataFrame.from_dict(mDict, orient='index')

In [ ]:
ids = ['BGE 144 IV 28','BGE 144 IV 1']
dates = ['1969-12-5', '1969-9-4']
relevancy = [10, 4]

itemsN = ["id", "date", "relevancy"]
nodes = []

itemsL = ["source", "target"]
links = []

for i, val in enumerate(ids):
    print(i, val)
    oj = {"id": val, "date":dates[i], "relevancy":relevancy[i]}
    nodes.append(oj)
    print(nodes)

    
data= {
    'nodes' : nodes,
    'links': 0
}

print(data)